In [19]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import pandas as pd
import polars as pl
import numpy as np

import matplotlib.pyplot as plt

import proyeccion_rdr.features as features

plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

# Asumiendo que es una ocupacion del 80% de camas
INDICE_OCUPACION_CAMAS = 1.25

# Agrega las columnas de poblacion de interes
COLUMNAS_POBLACION_INTERES = {str(i): i for i in range(2017, 2036)}

# Proyección de casos INT al 2035

In [21]:
# Lee los casos de todos los macroprocesos
RUTA_ARCHIVOS = "../data/interim/0.1_casos_teoricos_diagnosticos.xlsx"
_, casos_macroproceso = features.leer_casos_macroprocesos(RUTA_ARCHIVOS)

# Obtiene los casos hospitalizados
casos_hospitalizados = casos_macroproceso.query("tipo_paciente == 'hospitalizados'")

In [22]:
# Los diagnosticos que tienen un porcentaje de hosp. mayor a 0
diagnosticos_relevantes = list(casos_hospitalizados.index)

# Ademas, los diagnosticos que estan agrupados dentro de los que aportan a AC
# Solo deja los diagnosticos que esten agrupados
diagnosticos_a_reasignar = casos_hospitalizados.dropna(subset="Diagnosticos Contenidos")

# Separa la lista de diagnosticos en filas
diagnosticos_agrupados = diagnosticos_a_reasignar.explode("Diagnosticos Contenidos")
diagnosticos_agrupados = list(diagnosticos_agrupados["Diagnosticos Contenidos"].unique())

# Suma los diagnosticos relevantes y los agrupados
diagnosticos_a_buscar_en_egresos = sorted(diagnosticos_relevantes + diagnosticos_agrupados)

## Obtención de casos atendidos entre 2017 y 2020 para diagnósticos relevantes.

Estos se obtendrán del análisis de egresos hospitalarios a nivel país DEIS.

In [23]:
# Lee los egresos del Torax
egresos_rdr = pl.read_csv("../data/raw/4_egresos_deis/egresos_procesados_109101.csv")

En primer lugar, se filtrarán ambas bases de datos para solamente tener la información de los diagnósticos más relevantes para el INT. Luego de esto, se calculará la cantidad de egresos, dias de estada y pacientes para cada uno de los diagnósticos. Estos insumos serán utilizados para estimar la demanda.


In [24]:
# Filtra solamente los diagnosticos mas relevantes del Torax
egresos_mas_relevantes_rdr = egresos_rdr.filter(
    (pl.col("DIAG1").is_in(diagnosticos_a_buscar_en_egresos))
)

In [25]:
egresos_totales_rdr = egresos_rdr.to_pandas().groupby("ANO_EGRESO").size()
egresos_relevantes_rdr = egresos_mas_relevantes_rdr.to_pandas().groupby("ANO_EGRESO").size()
comparacion_egresos = pd.concat([egresos_totales_rdr, egresos_relevantes_rdr], axis=1)
comparacion_egresos["%"] = round((comparacion_egresos[1] / comparacion_egresos[0]) * 100, 2)

In [26]:
comparacion_egresos

,0,1,%
ANO_EGRESO,,,
2001.0,12891,3198,24.81
2002.0,12889,3244,25.17
2003.0,14451,3845,26.61
2004.0,14038,4087,29.11
2005.0,14361,3865,26.91
2006.0,14597,3683,25.23
2007.0,13367,3444,25.76
2008.0,12301,2808,22.83
2009.0,13524,3663,27.09


## Reasginación de diagnósticos y agrupación

In [27]:
for row in diagnosticos_a_reasignar.itertuples():
    diagnostico_nuevo = row[0]
    diagnosticos_antiguos = row[1]

    print(f"Cambiando {diagnosticos_antiguos} a {diagnostico_nuevo}")
    egresos_mas_relevantes_rdr = egresos_mas_relevantes_rdr.with_columns(
        pl.col("DIAG1").replace(diagnosticos_antiguos, diagnostico_nuevo)
    )

Cambiando ['A090', 'A099'] a AGTO
Cambiando ['D231', 'D233', 'D236', 'D367', 'L720'] a DQUI
Cambiando ['Q262', 'K914', 'K638'] a ENTN
Cambiando ['H652', 'H653', 'H654', 'H669'] a HOTI
Cambiando ['J353'] a J351
Cambiando ['J205', 'J208', 'J209'] a JBRO
Cambiando ['J210', 'J218', 'J219'] a JBRQ
Cambiando ['J121', 'J122', 'J123', 'J129', 'J159', 'J189'] a JNEU
Cambiando ['K352', 'K353', 'K358'] a KAPE
Cambiando ['K565', 'K650'] a KGAST
Cambiando ['M411', 'M410', 'M412', 'M414', 'T813', 'Q763'] a MESC
Cambiando ['K560', 'Q625', 'T222'] a OBSI
Cambiando ['K36X', 'K659'] a PERC
Cambiando ['Q390', 'Q391', 'Q393'] a QATR
Cambiando ['Q375', 'Q374', 'Q369', 'Q371', 'Q379', 'Q373', 'Q370', 'T813', 'Q355', 'Q372', 'Q359', 'Q378'] a QLAB
Cambiando ['S424', 'S524', 'S525', 'S528', 'S529'] a SFRA
Cambiando ['T213', 'T202', 'T212', 'T203', 'T293', 'T201', 'T273', 'T223', 'T222', 'T243', 'T280', 'T211', 'T242', 'T253', 'T232', 'T221', 'T241', 'T312', 'T291', 'T292', 'T252'] a TQUE


In [28]:
# Obtiene resumen de metricas para el Torax
metricas_diags_relevantes_rdr = (
    features.calcular_resumen_metricas_desagregadas_y_agrupadas_en_anios(
        egresos_mas_relevantes_rdr, 2019, 2023
    )
)

In [29]:
# Define las columnas que se quieran corregir
columna_dias_estada = ("agrupado_entre_2019_2023", "dias_estada_promedio")

## Estimación de casos Hospitalizados utilizando % de Hospitalización del INT

En este caso, se utilizará el % de hospitalización del INT de los pacientes ambulatorios.

In [30]:
# Se obtiene la cantidad de egresos que debiese tener el INT, asumiendo que trabajara con la misma
# eficiencia
egresos_estimados_rdr = casos_hospitalizados[COLUMNAS_POBLACION_INTERES.keys()].mul(
    metricas_diags_relevantes_rdr[("agrupado_entre_2019_2023", "egresos_por_paciente")], axis=0
)

# Se obtiene la cantidad de dias de estada que debiese el INT, asumiendo que trabajara con la misma
# eficiencia
dias_estada_estimados_rdr = egresos_estimados_rdr.mul(
    metricas_diags_relevantes_rdr[("agrupado_entre_2019_2023", "dias_estada_promedio")], axis=0
)

# Estima la cantidad de camas necesaarias por diagnostico
camas_estimadas_rdr = (dias_estada_estimados_rdr / 365.25) * INDICE_OCUPACION_CAMAS

# Calcula las camas totales necesarias
camas_totales_rdr = camas_estimadas_rdr.sum()

In [31]:
# Se quiere tener un 60% de UPC (1:2, UCI:UTI, y 40% de Medias)
FRACCION_UCI = 0.21
FRACCION_UTI = 0.42
FRACCION_MEDIAS = 0.37

# Agrega las camas necesarias para trasplantes a las del 2035
camas_al_2035 = (camas_totales_rdr["2035"])
# Obtiene la cantidad de camas UCI, UTI y Medias sin redondear
camas_uci = (camas_al_2035 * FRACCION_UCI)
camas_uti = (camas_al_2035 * FRACCION_UTI)
camas_medias = (camas_al_2035 * FRACCION_MEDIAS)

# Obtiene la cantidad de camas UCI, UTI y Medias redondeadas
camas_uci_redondeadas = round(camas_al_2035 * FRACCION_UCI)
camas_uti_redondeadas = round(camas_al_2035 * FRACCION_UTI)
camas_medias_redondeadas = round(camas_al_2035 * FRACCION_MEDIAS)
camas_al_2035_redondeadas = round(camas_al_2035)

In [32]:
print(f"> Camas 2035: {camas_al_2035:.2f}")
# print(f"> UCI: {camas_uci:.2f} camas")
# print(f"> UTI: {camas_uti:.2f} camas")
# print(f"> Medias: {camas_medias:.2f} camas")
print()
# print(f"> UCI Redondeadas: {camas_uci_redondeadas} camas")
# print(f"> UTI Redondeadas: {camas_uti_redondeadas} camas")
# print(f"> Medias Redondeadas: {camas_medias_redondeadas} camas")
print(f"> Total Redondeadas: {camas_al_2035_redondeadas} camas")

> Camas 2035: 182.13

> Total Redondeadas: 182 camas


## Obtener resumen para MINSAL

In [33]:
# Obtiene porcentaje de hospitalizados
porcentaje_hospitalizados = pd.read_excel(
    "../data/raw/3_incidencias_y_porcentajes_marcoprocesos/incidencias_y_prevalencias_RDR.xlsx"
)

porcentaje_hospitalizados["Diagnostico"] = (
    porcentaje_hospitalizados["Diagnostico"].str.split(" - ").str[0]
)

# Filtra los porcentajes de hospitalizados
porcentaje_hospitalizados = porcentaje_hospitalizados.set_index("Diagnostico")
porcentaje_hospitalizados = porcentaje_hospitalizados["Porcentaje Pacientes Hospitalizados"]
porcentaje_hospitalizados = porcentaje_hospitalizados.dropna()
porcentaje_hospitalizados = porcentaje_hospitalizados[porcentaje_hospitalizados > 0]

In [34]:
# Obtiene resumen para MINSAL
resumen_hosp = pd.DataFrame(
    {
        "porcentaje_hospitalizado": porcentaje_hospitalizados,
        "casos_hospitalizados_2035": casos_hospitalizados["2035"],
        "egresos_por_paciente_agrupado_2019_2023": metricas_diags_relevantes_rdr[
            ("agrupado_entre_2019_2023", "egresos_por_paciente")
        ],
        "egresos_2035": egresos_estimados_rdr["2035"],
        "dias_estada_promedio_agrupado_2019_2023": metricas_diags_relevantes_rdr[
            ("agrupado_entre_2019_2023", "dias_estada_promedio")
        ],
        "dias_estada_totales_2035": dias_estada_estimados_rdr["2035"],
        "camas_totales_2035": camas_estimadas_rdr["2035"],
    }
)

Finalmente, une el resumen de casos teoricos desde el cuadernillo 1.0 con el resumen de este cuadernillo. Esto obtiene la tabla final para MINSAL.

In [35]:
resumen_casos_teoricos = pd.read_excel(
    "../data/interim/0.1_casos_teoricos_diagnosticos.xlsx", sheet_name="resumen_total_RDR"
)

resumen_casos_teoricos["Diagnostico_codigo"] = (
    resumen_casos_teoricos["Diagnostico"].str.split(" - ").str[0]
)
resumen_casos_teoricos = resumen_casos_teoricos.set_index("Diagnostico_codigo")

resumen_minsal = resumen_casos_teoricos.merge(
    resumen_hosp, how="right", left_index=True, right_index=True
)

resumen_minsal = resumen_minsal.reset_index(drop=True).set_index("Diagnostico")

## Guardar archivos

In [36]:
archivos_a_guardar = {
    "resumen_MINSAL": resumen_minsal,
    "metricas_relevantes_RDR": metricas_diags_relevantes_rdr,
    "casos_hospitalizados_RDR": casos_hospitalizados,
    "egresos_estimados_RDR": egresos_estimados_rdr,
    "dias_estada_estimados_RDR": dias_estada_estimados_rdr,
    "camas_estimadas_desglosadas_RDR": camas_estimadas_rdr,
    "camas_totales_RDR": camas_totales_rdr,
    "resumen_total_hosp_RDR": resumen_hosp,
}

with pd.ExcelWriter("../data/interim/1.0_estimacion_atencion_cerrada_RDR.xlsx") as file:
    for nombre_hoja, df_a_guardar in archivos_a_guardar.items():
        print(f"Guardando {nombre_hoja}")
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja)

Guardando resumen_MINSAL
Guardando metricas_relevantes_RDR
Guardando casos_hospitalizados_RDR
Guardando egresos_estimados_RDR
Guardando dias_estada_estimados_RDR
Guardando camas_estimadas_desglosadas_RDR
Guardando camas_totales_RDR
Guardando resumen_total_hosp_RDR
